In [3]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 4.0 MB/s eta 0:00:00


In [4]:
!pip install colorama


In [14]:
from groq import Groq

# --- Setup ---
api_key = input(" Enter your Groq API Key: ")
llm = Groq(api_key=api_key)

# Medical system prompt
system_prompt = """You are a professional virtual medical assistant.
You provide accurate, safe, and step-by-step explanations for medical questions.
Always include:
1. A clear diagnosis or explanation.
2. Possible causes.
3. Recommended next steps or precautions.
4. A disclaimer that this is not a substitute for a doctor's visit.
"""

continue_chat = True

while continue_chat:
    print("\n--- Session Settings ---")
    temperature = 0.5
    mode = input(" Choose mode [0] Zero-shot:if you want Yes/No answer  or [1] One-shot: If you want a detailed answer")

    user_input = input(" Enter your medical question: ")

    # Prepare messages
    messages = [{"role": "system", "content": system_prompt}]

    if mode == "1":  # One-shot example
        example_question = {
            "role": "user",
            "content": "I have a sore throat and mild fever. What could be the reason?",
        }
        example_answer = {
            "role": "assistant",
            "content": "It could be a viral infection like the common cold or flu. Causes include viruses, bacteria, or allergies. Stay hydrated, rest, and consult a doctor if symptoms worsen. This is not a substitute for professional medical advice."
        }
        messages.append(example_question)
        messages.append(example_answer)

    messages.append({"role": "user", "content": user_input})

    # Call the LLM
    response = llm.chat.completions.create(
        model="llama3-8b-8192",  # Supported model
        messages=messages,
        temperature=temperature,
        max_tokens=500
    )

    # Print the reply
    if mode == "0":  # Zero-shot -> simple Yes/No
        print("\n AI Medical Assistant:\n", "Yes" if "yes" in response.choices[0].message.content.lower() else "No")
    else:
        reply = response.choices[0].message.content
        print("\n AI Medical Assistant:\n", reply)

    # Ask if the user wants another question
    ask_again = input("\n Do you want to ask another question? (yes/no): ").strip().lower()
    if ask_again != "yes":
        continue_chat = False
        print(" Goodbye!")


 Enter your Groq API Key: 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

--- Session Settings ---
 Choose mode [0] Zero-shot:if you want Yes/No answer  or [1] One-shot: If you want a detailed answer0
 Enter your medical question: Can I take ibuprofen and paracetamol together?

 AI Medical Assistant:
 No

 Do you want to ask another question? (yes/no): yes

--- Session Settings ---
 Choose mode [0] Zero-shot:if you want Yes/No answer  or [1] One-shot: If you want a detailed answer1
 Enter your medical question: Can I take ibuprofen and paracetamol together?

 AI Medical Assistant:
 I must advise against taking ibuprofen and paracetamol together without consulting a doctor. Both medications can cause stomach upset, bleeding, and kidney damage when taken together. Ibuprofen is a non-steroidal anti-inflammatory drug (NSAID) and paracetamol is an acetaminophen. Taking both can increase the risk of adverse effects.

Instead, I recommend speaking with your doctor or pharmacist to d

In [18]:
import gradio as gr
from groq import Groq

# --- Setup Groq LLM ---
api_key = input("Enter your Groq API Key: ")
llm = Groq(api_key=api_key)

system_prompt = """You are a professional virtual medical assistant.
You provide accurate, safe, and step-by-step explanations for medical questions.
Always include:
1. A clear diagnosis or explanation.
2. Possible causes.
3. Recommended next steps or precautions.
4. A disclaimer that this is not a substitute for a doctor's visit.
"""

TEMPERATURE = 0.5  # Fixed temperature

def medical_chat(user_input, mode):
    messages = [{"role": "system", "content": system_prompt}]

    if mode == "One-shot":
        example_question = {"role": "user", "content": "I have a sore throat and mild fever. What could be the reason?"}
        example_answer = {"role": "assistant", "content": "It could be a viral infection like the common cold or flu. Causes include viruses, bacteria, or allergies. Stay hydrated, rest, and consult a doctor if symptoms worsen. This is not a substitute for professional medical advice."}
        messages.append(example_question)
        messages.append(example_answer)

    # For Zero-shot, force Yes/No response
    if mode == "Zero-shot":
        messages.append({"role": "system", "content": "Answer the user's question with only 'Yes' or 'No', no explanation."})

    messages.append({"role": "user", "content": user_input})

    response = llm.chat.completions.create(
        model="llama3-8b-8192",
        messages=messages,
        temperature=TEMPERATURE,
        max_tokens=500
    )

    # Return Yes/No for Zero-shot, full response for One-shot
    if mode == "Zero-shot":
        answer_text = response.choices[0].message.content.strip().lower()
        return "Yes" if "yes" in answer_text else "No"
    else:
        return response.choices[0].message.content

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("##  AI Medical Assistant Chatbot")
    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(label="Enter your medical question")
            mode = gr.Radio(choices=["Zero-shot", "One-shot"], label="Mode")
            submit_btn = gr.Button("Ask")
        with gr.Column():
            output = gr.Textbox(label="AI Response", lines=20)

    submit_btn.click(medical_chat, inputs=[user_input, mode], outputs=[output])

demo.launch()


Enter your Groq API Key: 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
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7f949da6f357da2413.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
